In [ ]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
%matplotlib notebook

In [ ]:
f = h5py.File('edep-sim_test.h5','r')

In [ ]:
my_func = lambda name,dset : print(name) if isinstance(dset, h5py.Dataset) else None
f.visititems(my_func) # see which datasets are available

In [ ]:
# load edep-sim h5 file
f = h5py.File('edep-sim_file.h5','r')

In [ ]:
# print names of all variables available in a dataset
print(f['segments'].dtype.names)
print(" ")
print(f['trajectories'].dtype.names)

In [ ]:
# selecting single event
evtID = 0
event = f['segments'][f['segments']['eventID'] == evtID]

In [ ]:
# select main track (no secondaries)
tracknum = 0
event = f['segments'][f['segments']['trackID'] == tracknum]

In [ ]:
# select single event and main track
evtID = 0
tracknum = 0
event = f['segments'][f['segments']['eventID'] == evtID]
event_track = event[event['trackID'] == tracknum]

You can sort through a dataset like segments to find the data you want. 
As a first example, you could find all the zeroth segments of the events and
put them into an array. Here I use a list comprehension, but you could turn it into
a normal for loop.

In [ ]:
events = f['segments']
evt_tot = max(f['segments']['eventID'])+1
segs0 = np.array([ events[events['eventID'] == i][0] for i in range(evt_tot) ])

In [1]:
# here's an example of a simple list comprehension if you're not familiar
[i for i in range(5)]

[0, 1, 2, 3, 4]

You might want to do a loop through the data if there isn't a way to use numpy array slicing to find what you want (or if you're me, you just can't think of how to do it).
This, however, can get very slow sometimes. There may be quicker ways to do things like this but I'm not personally too familiar.

I have, however, noticed a way around making such searches slow. Recall that dumpTree.py is already looping through the edep-sim ROOT tree data to make the h5 file. So what you can do is make a modified version of dumpTree.py that keeps track of what you want. It may take some scratching your head to understand how dumpTree.py works, but you only need a modest amount of understanding of it. You can even create your own datasets like segments and trajectories. I've found that this is immensely faster than looping through the datasets in python. Another possible alternative is to loop through the data with a faster language like C++.

Here's an example of plotting edep-sim segments, event by event.

In [2]:
def plotEvent(evNum):
    # function for plotting the segments of a single event
    # INPUT: eventID number
    # OUTPUT: Nothing just plots.
    
    boolArr = f['segments']['eventID'] == evNum
    event = f['segments'][boolArr]
    #len(event)

    x_start = event['x_start']
    x_end = event['x_end']
    y_start = event['y_start']
    y_end = event['y_end']
    z_start = event['z_start']
    z_end = event['z_end']

    x_mid = (x_start+x_end)/2
    y_mid = (y_start+y_end)/2
    z_mid = (z_start+z_end)/2

    segmentlengths = np.sqrt((x_start - x_end)**2 + (y_start - y_end)**2 + (z_start - z_end)**2)
    print('Total track length = ', sum(segmentlengths) * 10, ' mm')

    for i in range(len(event)):
        if event['pdgId'][i] == 11 and event['trackID'][i] == 0:
            plt.plot([x_start[i],x_end[i]],[z_start[i],z_end[i]], [y_start[i], y_end[i]], 'b-')
        elif event['pdgId'][i] == 11 and event['trackID'][i] != 0:
            print('Here!')
            plt.plot([x_start[i],x_end[i]],[z_start[i],z_end[i]], [y_start[i], y_end[i]], 'r-')
        elif event['pdgId'][i] == 22 and event['trackID'][i] != 0:
            print('Here2!')
            plt.plot([x_start[i],x_end[i]],[z_start[i],z_end[i]], [y_start[i], y_end[i]], 'g-')
            #if i < 5:
            #    plt.plot([x_start[i],x_end[i]],[z_start[i],z_end[i]], [y_start[i], y_end[i]], 'g-')
            #elif i >= 5:
            #    plt.plot([x_start[i],x_end[i]],[z_start[i],z_end[i]], [y_start[i], y_end[i]], 'b-')
        #if event['pdgId'][i] != 11:
        #    plt.plot([x_start[i],x_end[i]],[z_start[i],z_end[i]], [y_start[i], y_end[i]], 'r-')


In [ ]:
# create figure. I only tested this plotting with %matplotlib notebook, where it is continuously updated
# with segments. So I'm not sure if it works as is with out that.
fig = plt.figure()
ax = fig.add_subplot(projection='3d')

ax.set_xlabel('x [cm]')
ax.set_ylabel('z [cm]')
ax.set_zlabel('y [cm]')

## settings to use for module 0 if you want to show the whole detector. Otherwise the figure expands to
## the size of the tracks you plot

#ax.set_xlim(-30,30)
#ax.set_ylim(-30,30)
#ax.set_zlim(-80, 40)
#ax.set_box_aspect([1,1,2])

In [ ]:
eventtoplot = 0
plotEvent(eventtoplot)